In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras import metrics
from sklearn.model_selection import train_test_split

In [2]:
file_paths = [
    'blackhole.csv',
    'dodag.csv',
    'flooding.csv',
    'rank.csv'
]

In [3]:
# Load all CSV files into a list of DataFrames
dfs = [pd.read_csv(file) for file in file_paths]

In [4]:
# Concatenate all DataFrames into a single DataFrame
data = pd.concat(dfs,ignore_index=True)

In [5]:
# Display the first few rows
print("Dataset Preview:")
print(data.head())

Dataset Preview:
    time  source  destination  length  info  transmission_rate_per_1000_ms  \
0  0.037      39         9999     0.0   1.0                       0.000000   
1  0.037      39         9999     0.0   1.0                       0.000000   
2  0.038      39         9999     0.0   1.0                       0.671176   
3  0.045      39         9999     0.0   1.0                       0.000000   
4  0.046      39         9999     0.0   1.0                       0.000000   

   reception_rate_per_1000_ms  transmission_average_per_sec  \
0                    0.671176                      0.000000   
1                    0.649873                      0.000000   
2                    0.652361                      0.462516   
3                    0.633786                      0.000000   
4                    0.630378                      0.000000   

   reception_average_per_sec  transmission_count_per_sec  \
0                   0.499879                    0.000000   
1              

In [6]:
# Basic dataset information
print("\nDataset Info:")
print(data.info())


Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1639975 entries, 0 to 1639974
Data columns (total 18 columns):
 #   Column                               Non-Null Count    Dtype  
---  ------                               --------------    -----  
 0   time                                 1639975 non-null  float64
 1   source                               1639975 non-null  int64  
 2   destination                          1639975 non-null  int64  
 3   length                               1639975 non-null  float64
 4   info                                 1639975 non-null  float64
 5   transmission_rate_per_1000_ms        1639975 non-null  float64
 6   reception_rate_per_1000_ms           1639975 non-null  float64
 7   transmission_average_per_sec         1639975 non-null  float64
 8   reception_average_per_sec            1639975 non-null  float64
 9   transmission_count_per_sec           1639975 non-null  float64
 10  reception_count_per_sec              1639975 non-nu

In [7]:
# Check for missing values
print("\nMissing Values (Before Handling):")
print(data.isnull().sum())


Missing Values (Before Handling):
time                                   0
source                                 0
destination                            0
length                                 0
info                                   0
transmission_rate_per_1000_ms          0
reception_rate_per_1000_ms             0
transmission_average_per_sec           0
reception_average_per_sec              0
transmission_count_per_sec             0
reception_count_per_sec                0
transmission_total_duration_per_sec    0
reception_total_duration_per_sec       0
dao                                    0
dis                                    0
dio                                    0
category                               0
label                                  0
dtype: int64


In [8]:
data.nunique().sort_values()

label                                       2
category                                    5
info                                       10
length                                     13
destination                               101
source                                    101
dis                                       206
dao                                       233
dio                                       383
transmission_count_per_sec                467
transmission_rate_per_1000_ms             470
reception_total_duration_per_sec          575
reception_rate_per_1000_ms               1248
reception_count_per_sec                  1253
transmission_total_duration_per_sec      1634
reception_average_per_sec                9369
transmission_average_per_sec            21764
time                                   725703
dtype: int64

In [9]:
print(data['info'].sort_values().unique())

[0.         0.41111834 0.52029776 0.55440486 0.57018941 0.57241282
 0.57289431 0.58018146 0.58259798 1.        ]


In [10]:
print(data['length'].sort_values().unique())

[0.         0.46394382 0.483634   0.48436797 0.48546724 0.48595518
 0.4943263  0.50241319 0.58086668 0.58814455 0.59556985 0.6370377
 1.        ]


In [11]:
print(data['source'].sort_values().unique())

[  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
  91  92  93  94  95  96  97  98  99 100 101]


In [12]:
print(data['destination'].sort_values().unique())

[   1    2    3    4    5    6    7    8    9   10   11   12   13   14
   15   16   17   18   19   20   21   22   23   24   25   26   27   28
   29   30   31   32   33   34   35   36   37   38   39   40   41   42
   43   44   45   46   47   48   49   50   51   52   53   54   55   56
   57   58   59   60   61   62   63   64   65   66   67   68   69   70
   71   72   73   74   75   76   77   78   79   80   81   82   83   84
   85   86   87   88   89   90   91   92   93   94   96   97   98   99
  100  101 9999]


In [13]:
import tensorflow as tf
from tensorflow.keras import layers

In [14]:
source_lookup = tf.keras.layers.IntegerLookup(output_mode='int')
destination_lookup = tf.keras.layers.IntegerLookup(output_mode='int')
info_lookup = tf.keras.layers.StringLookup(output_mode='int')
length_lookup = tf.keras.layers.StringLookup(output_mode='int')

source_lookup.adapt(data['source'])
destination_lookup.adapt(data['destination'])
info_lookup.adapt(data['info'].astype(str))
length_lookup.adapt(data['length'].astype(str))


In [15]:
layers_embedding_dim = 8

source_embedding_layer = layers.Embedding(input_dim=source_lookup.vocabulary_size(), output_dim= layers_embedding_dim)
source_input = tf.keras.Input(shape=(1,), dtype=tf.int64, name='source')
source_index = source_lookup(source_input)
source_embedding = source_embedding_layer(source_index)

destination_embedding_layer = layers.Embedding(input_dim=destination_lookup.vocabulary_size(), output_dim= layers_embedding_dim)
destination_input = tf.keras.Input(shape=(1,), dtype=tf.int64, name= 'destination')
destination_index = destination_lookup(destination_input)
destination_embedding = destination_embedding_layer(destination_index)

info_embedding_layer = layers.Embedding(input_dim=info_lookup.vocabulary_size(), output_dim= layers_embedding_dim)
info_input = tf.keras.Input(shape=(1,), dtype=tf.string, name='info')
info_index = info_lookup(info_input)
info_embedding = info_embedding_layer(info_index)

length_embedding_layer = layers.Embedding(input_dim=length_lookup.vocabulary_size(), output_dim= layers_embedding_dim)
length_input = tf.keras.Input(shape=(1,), dtype=tf.string, name='length')
length_index = length_lookup(length_input)
length_embedding = length_embedding_layer(length_index)

In [16]:
# Convert a value to index in vocabulary
# value = tf.constant(39)
# print(type(value))
# index = source_lookup(value)

# # Get corresponding Embedding vector  
# embedding_vector = source_embedding_layer(index)
# print(embedding_vector)

In [17]:
# Convert a value to index in vocabulary
# value = tf.constant("0.58018146")
# print(type(value))
# index = info_lookup(value)

# # Get corresponding Embedding vector  
# embedding_vector = info_embedding_layer(index)
# print(embedding_vector)

In [18]:
# Numerical -> Vector
numerical_vector_dim = 8
"""
    DIS
"""
dis_norm_layer = tf.keras.layers.Normalization()
dis_norm_layer.adapt(data['dis'].values.reshape(-1, 1))

dis_input = tf.keras.Input(shape=(1,), dtype=tf.float64, name='dis')
dis_norm = dis_norm_layer(dis_input)
dis_vector = layers.Dense(numerical_vector_dim)(dis_norm)
dis_vector = layers.Reshape((1, numerical_vector_dim))(dis_vector)

"""
    DAO
"""
dao_norm_layer = tf.keras.layers.Normalization()
dao_norm_layer.adapt(data['dao'].values.reshape(-1, 1))

dao_input = tf.keras.Input(shape=(1,), dtype=tf.float64, name='dao')
dao_norm = dao_norm_layer(dao_input)
dao_vector = layers.Dense(numerical_vector_dim)(dao_norm)
dao_vector = layers.Reshape((1, numerical_vector_dim))(dao_vector)

"""
    DIO feature
"""
dio_norm_layer = tf.keras.layers.Normalization()
dio_norm_layer.adapt(data['dio'].values.reshape(-1, 1))

dio_input = tf.keras.Input(shape=(1,), dtype=tf.float64, name='dio')
dio_norm = dio_norm_layer(dio_input)
dio_vector = layers.Dense(numerical_vector_dim)(dio_norm)
dio_vector = layers.Reshape((1, numerical_vector_dim))(dio_vector)

"""
    transmission_count_per_sec feature
"""
transmission_count_per_sec_norm_layer = tf.keras.layers.Normalization()
transmission_count_per_sec_norm_layer.adapt(data['transmission_count_per_sec'].values.reshape(-1, 1))


transmission_count_per_sec_input = tf.keras.Input(shape=(1,), dtype=tf.float64, name='transmission_count_per_sec')
transmission_count_per_sec_norm = transmission_count_per_sec_norm_layer(transmission_count_per_sec_input)
transmission_count_per_sec_vector = layers.Dense(numerical_vector_dim)(transmission_count_per_sec_norm)
transmission_count_per_sec_vector = layers.Reshape((1, numerical_vector_dim))(transmission_count_per_sec_vector)

"""
    transmission_rate_per_1000_ms feature
"""

transmission_rate_per_1000_ms_norm_layer = tf.keras.layers.Normalization()
transmission_rate_per_1000_ms_norm_layer.adapt(data['transmission_rate_per_1000_ms'].values.reshape(-1, 1))

transmission_rate_per_1000_ms_input = tf.keras.Input(shape=(1,), dtype=tf.float64, name='transmission_rate_per_1000_ms')
transmission_rate_per_1000_ms_norm = transmission_rate_per_1000_ms_norm_layer(transmission_rate_per_1000_ms_input)
transmission_rate_per_1000_ms_vector = layers.Dense(numerical_vector_dim)(transmission_rate_per_1000_ms_norm)
transmission_rate_per_1000_ms_vector = layers.Reshape((1, numerical_vector_dim))(transmission_rate_per_1000_ms_vector)

"""
    time_input feature
"""
time_input_norm_layer = tf.keras.layers.Normalization()
time_input_norm_layer.adapt(data['time'].values.reshape(-1, 1))

time_input = tf.keras.Input(shape=(1,), dtype=tf.float64, name='time')
time_norm = time_input_norm_layer(time_input)
time_vector = layers.Dense(numerical_vector_dim)(time_norm)
time_vector = layers.Reshape((1, numerical_vector_dim))(time_vector)

"""
    reception_total_duration_per_sec feature 
"""
reception_total_duration_per_sec_norm_layer = tf.keras.layers.Normalization()
reception_total_duration_per_sec_norm_layer.adapt(data['reception_total_duration_per_sec'].values.reshape(-1, 1))

reception_total_duration_per_sec_input = tf.keras.Input(shape=(1,), dtype=tf.float64, name='reception_total_duration_per_sec')
reception_total_duration_per_sec_norm = reception_total_duration_per_sec_norm_layer(reception_total_duration_per_sec_input)
reception_total_duration_per_sec_vector = layers.Dense(numerical_vector_dim)(reception_total_duration_per_sec_norm)
reception_total_duration_per_sec_vector = layers.Reshape((1, numerical_vector_dim))(reception_total_duration_per_sec_vector)

"""
    reception_rate_per_1000_ms feature 
"""
reception_rate_per_1000_ms_norm_layer = tf.keras.layers.Normalization()
reception_rate_per_1000_ms_norm_layer.adapt(data['reception_rate_per_1000_ms'].values.reshape(-1, 1))

reception_rate_per_1000_ms_input = tf.keras.Input(shape=(1,), dtype=tf.float64, name='reception_rate_per_1000_ms')
reception_rate_per_1000_ms_norm = reception_rate_per_1000_ms_norm_layer(reception_rate_per_1000_ms_input)
reception_rate_per_1000_ms_vector = layers.Dense(numerical_vector_dim)(reception_rate_per_1000_ms_norm)
reception_rate_per_1000_ms_vector = layers.Reshape((1, numerical_vector_dim))(reception_rate_per_1000_ms_vector)

"""
    reception_count_per_sec feature
"""
reception_count_per_sec_norm_layer = tf.keras.layers.Normalization()
reception_count_per_sec_norm_layer.adapt(data['reception_count_per_sec'].values.reshape(-1,1))

reception_count_per_sec_input = tf.keras.Input(shape=(1,), dtype=tf.float64, name='reception_count_per_sec')
reception_count_per_sec_norm = reception_count_per_sec_norm_layer(reception_count_per_sec_input)
reception_count_per_sec_vector = layers.Dense(numerical_vector_dim)(reception_count_per_sec_norm)
reception_count_per_sec_vector = layers.Reshape((1, numerical_vector_dim))(reception_count_per_sec_vector)

"""
    transmission_total_duration_per_sec feature
"""
transmission_total_duration_per_sec_norm_layer = tf.keras.layers.Normalization()
transmission_total_duration_per_sec_norm_layer.adapt(data['transmission_total_duration_per_sec'].values.reshape(-1,1))

transmission_total_duration_per_sec_input = tf.keras.Input(shape=(1,), dtype=tf.float64, name='transmission_total_duration_per_sec')
transmission_total_duration_per_sec_norm = transmission_total_duration_per_sec_norm_layer(transmission_total_duration_per_sec_input)
transmission_total_duration_per_sec_vector = layers.Dense(numerical_vector_dim)(transmission_total_duration_per_sec_norm)
transmission_total_duration_per_sec_vector = layers.Reshape((1, numerical_vector_dim))(transmission_total_duration_per_sec_vector)

"""
    reception_average_per_sec feature
"""
reception_average_per_sec_norm_layer = tf.keras.layers.Normalization()
reception_average_per_sec_norm_layer.adapt(data['reception_average_per_sec'].values.reshape(-1,1))

reception_average_per_sec_input = tf.keras.Input(shape=(1,), dtype=tf.float64, name='reception_average_per_sec')
reception_average_per_sec_norm = reception_average_per_sec_norm_layer(reception_average_per_sec_input)
reception_average_per_sec_vector = layers.Dense(numerical_vector_dim)(reception_average_per_sec_norm)
reception_average_per_sec_vector = layers.Reshape((1, numerical_vector_dim))(reception_average_per_sec_vector)

"""
    transmission_average_per_sec feature
"""
transmission_average_per_sec_norm_layer = tf.keras.layers.Normalization()
transmission_average_per_sec_norm_layer.adapt(data['transmission_average_per_sec'].values.reshape(-1,1))

transmission_average_per_sec_input = tf.keras.Input(shape=(1,), dtype=tf.float64, name='transmission_average_per_sec')
transmission_average_per_sec_norm = transmission_average_per_sec_norm_layer(transmission_average_per_sec_input)
transmission_average_per_sec_vector = layers.Dense(numerical_vector_dim)(transmission_average_per_sec_norm)
transmission_average_per_sec_vector = layers.Reshape((1, numerical_vector_dim))(transmission_average_per_sec_vector)


In [19]:
print(dao_vector.shape)
print(transmission_average_per_sec_vector.shape)

(None, 1, 8)
(None, 1, 8)


In [ ]:
X = data.drop(columns='label')
X = X.drop(columns='category')
y = data['label'].values

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2 )

In [ ]:
x_train = {
    col: (
        X_train[col].values.astype('<U32').reshape(-1, 1) if col in ['info', 'length']
        else X_train[col].values.reshape(-1, 1)
    )
    for col in X_train.columns
}

In [ ]:
for col in ['info', 'length']:
    x_train[col] = tf.constant(x_train[col].flatten(), dtype=tf.string)
    x_train[col] = tf.reshape(x_train[col], (-1, 1))

In [20]:
concatenated = layers.Concatenate(axis=1)([time_vector, 
                                           source_embedding, 
                                           destination_embedding, 
                                           length_embedding, 
                                           info_embedding,
                                           transmission_rate_per_1000_ms_vector,
                                           reception_rate_per_1000_ms_vector,
                                           transmission_average_per_sec_vector,
                                           reception_average_per_sec_vector,
                                           transmission_count_per_sec_vector,
                                           reception_count_per_sec_vector,
                                           transmission_total_duration_per_sec_vector,
                                           reception_total_duration_per_sec_vector,
                                           dao_vector,
                                           dis_vector,
                                           dio_vector])
print(concatenated.shape)

(None, 16, 8)


In [21]:
def transformer_block(x, num_heads=2, key_dim=4, ff_dim=16, dropout=0.1):
    # Multi-head attention
    attn_output = tf.keras.layers.MultiHeadAttention(num_heads=num_heads,
                                                      key_dim=key_dim)(x, x)
    attn_output = tf.keras.layers.Dropout(dropout)(attn_output)
    out1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)(x + attn_output)

    # Feed-forward layer
    ffn = tf.keras.Sequential([
        tf.keras.layers.Dense(ff_dim, activation="relu"),
        tf.keras.layers.Dense(x.shape[-1]),
    ])
    ffn_output = ffn(out1)
    ffn_output = tf.keras.layers.Dropout(dropout)(ffn_output)
    return tf.keras.layers.LayerNormalization(epsilon=1e-6)(out1 + ffn_output)

In [22]:

x = transformer_block(concatenated)
# x = transformer_block(x)
x = tf.keras.layers.GlobalAveragePooling1D()(x)
output = layers.Dense(1, activation='sigmoid')(x)

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping

# =========================
# 1. Lịch học rate: Warmup + Cosine Decay
# =========================
class WarmupCosineDecay(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, base_lr, batch_size, reference_batch_size, total_steps, warmup_steps, min_lr=1e-6):
        super().__init__()
        self.target_lr = base_lr * (batch_size / reference_batch_size)
        self.total_steps = total_steps
        self.warmup_steps = warmup_steps
        self.min_lr = min_lr

    def __call__(self, step):
        # Phase 1: warmup (tăng tuyến tính)
        if self.warmup_steps > 0:
            warmup_lr = self.base_lr * tf.cast(step, tf.float32) / tf.cast(self.warmup_steps, tf.float32)
        else:
            warmup_lr = self.base_lr

        # Phase 2: cosine decay
        cosine_decay = 0.5 * (1 + tf.cos(
            tf.constant(tf.constant(3.141592653589793)) * 
            (tf.cast(step - self.warmup_steps, tf.float32) / tf.cast(self.total_steps - self.warmup_steps, tf.float32))
        ))
        decayed = (self.base_lr - self.min_lr) * cosine_decay + self.min_lr

        return tf.cond(step < self.warmup_steps,
                       lambda: warmup_lr,
                       lambda: decayed)

In [ ]:
# =========================
# Setup Optimizer AdamW + LR schedule
# =========================
epochs = 100
batch_size = 1024
reference_batch_size = 32
steps_per_epoch = y_train.shape[0] // batch_size  # ví dụ, = len(train_data)//batch_size
total_steps = epochs * steps_per_epoch
warmup_steps = int(0.1 * total_steps)  # warmup 10%

base_lr = 1e-3
lr_schedule = WarmupCosineDecay(base_lr, batch_size, reference_batch_size, total_steps, warmup_steps, min_lr=1e-6)

optimizer = tf.keras.optimizers.AdamW(
    learning_rate=lr_schedule,
    weight_decay=1e-4
)

In [ ]:
inputs=[    
    time_input, 
    source_input, 
    destination_input, 
    length_input, 
    info_input,
    transmission_rate_per_1000_ms_input,
    reception_rate_per_1000_ms_input,
    transmission_average_per_sec_input,
    reception_average_per_sec_input,
    transmission_count_per_sec_input,
    reception_count_per_sec_input,
    transmission_total_duration_per_sec_input,
    reception_total_duration_per_sec_input,
    dao_input,
    dis_input,
    dio_input]
model = tf.keras.Model(inputs=inputs, outputs=output)
model.compile(optimizer= optimizer, loss='binary_crossentropy', metrics=['accuracy', metrics.Precision(name='precision'), metrics.Recall(name='recall')])


In [24]:
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ time (InputLayer)   │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ transmission_rate_… │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reception_rate_per… │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ transmission_avera… │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reception_average_… │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ transmission_count… │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reception_count_pe… │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ transmission_total… │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reception_total_du… │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dao (InputLayer)    │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dis (InputLayer)    │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dio (InputLayer)    │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalization_5     │ (None, 1)         │          3 │ time[0][0]        │
│ (Normalization)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ source (InputLayer) │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ destination         │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ length (InputLayer) │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ info (InputLayer)   │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalization_4     │ (None, 1)         │          3 │ transmission_rat… │
│ (Normalization)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalization_7     │ (None, 1)         │          3 │ reception_rate_p… │
│ (Normalization)     │                   │            │                 

 Total params: 2,669 (10.47 KB)

 Trainable params: 2,633 (10.29 KB)

 Non-trainable params: 36 (192.00 B)

In [ ]:
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, Callback

# # Callback custom: dừng khi thấy overfitting rõ rệt
# class OverfitStop(Callback):
#     def __init__(self, patience=3, gap_threshold=0.05):
#         super(OverfitStop, self).__init__()
#         self.patience = patience
#         self.gap_threshold = gap_threshold
#         self.wait = 0

#     def on_epoch_end(self, epoch, logs=None):
#         logs = logs or {}
#         train_loss = logs.get("loss")
#         val_loss = logs.get("val_loss")

#         if val_loss and train_loss and (val_loss - train_loss) > self.gap_threshold:
#             self.wait += 1
#             if self.wait >= self.patience:
#                 print(f"\n⚠️ Dừng sớm vì phát hiện overfitting tại epoch {epoch+1}")
#                 self.model.stop_training = True
#         else:
#             self.wait = 0


def get_callbacks(model_path="best_model.keras"):
    # 1. Lưu model tốt nhất
    checkpoint = ModelCheckpoint(
        model_path,
        monitor="val_loss",
        save_best_only=True,
        mode="min",
        verbose=1
    )

    # 2. EarlyStopping
    early_stop = EarlyStopping(
        monitor="val_loss",
        patience=3,  # số epoch chờ trước khi dừng
        restore_best_weights=True,
        verbose=1
    )

    return [checkpoint, early_stop]

In [32]:
callbacks = get_callbacks("best_model.keras")

In [ ]:
history = model.fit(
    x_train, y_train,
    epochs=100,
    validation_split=0.2,   # lấy 20% dữ liệu train làm validation
    callbacks=callbacks,
    verbose=1
)

Epoch 1/100
16395/16400 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8908 - loss: 0.2510 - precision: 0.8432 - recall: 0.8437
Epoch 1: val_loss improved from inf to 0.07744, saving model to best_model.keras
16400/16400 ━━━━━━━━━━━━━━━━━━━━ 166s 10ms/step - accuracy: 0.8908 - loss: 0.2510 - precision: 0.8432 - recall: 0.8437 - val_accuracy: 0.9728 - val_loss: 0.0774 - val_precision: 0.9580 - val_recall: 0.9656 - learning_rate: 0.0010
Epoch 2/100
16397/16400 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9666 - loss: 0.0896 - precision: 0.9520 - recall: 0.9536
Epoch 2: val_loss improved from 0.07744 to 0.06503, saving model to best_model.keras
16400/16400 ━━━━━━━━━━━━━━━━━━━━ 164s 10ms/step - accuracy: 0.9666 - loss: 0.0896 - precision: 0.9520 - recall: 0.9536 - val_accuracy: 0.9761 - val_loss: 0.0650 - val_precision: 0.9458 - val_recall: 0.9893 - learning_rate: 0.0010
Epoch 3/100
16399/16400 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9756 - loss: 0.0654 - precision: 0.9640 - reca

In [ ]:
from tensorflow.keras.models import load_model
best_model = load_model("best_model.keras")

In [ ]:
x_test = {
    col: (
        X_test[col].values.astype('<U32').reshape(-1, 1) if col in ['info', 'length']
        else X_test[col].values.reshape(-1, 1)
    )
    for col in X_test.columns
}

In [ ]:
for col in ['info', 'length']:
    x_test[col] = tf.constant(x_test[col].flatten(), dtype=tf.string)
    x_test[col] = tf.reshape(x_test[col], (-1, 1))

In [ ]:
print(type(y_test))
count = np.sum(y_test == 0)
print(count)

<class 'numpy.ndarray'>
211835


In [ ]:
best_model.evaluate(x_test, y_test)

10250/10250 ━━━━━━━━━━━━━━━━━━━━ 26s 2ms/step - accuracy: 0.9888 - loss: 0.0297 - precision: 0.9781 - recall: 0.9904


[0.029438022524118423,
 0.9888535141944885,
 0.97847980260849,
 0.9903064966201782]